# Control Variates for Monte Carlo Variance Reduction
## Kai Chang

In this write-up, following the `mc.ipynb`, we consider applying several advanced variance-reducing techniques for Monte Carlo simulations. We apply these schemes in the context of simulating a stochastic differential equation.

In [1]:
import numpy as np
from numpy.random import normal 

In [2]:
"""
Solve 1-D diffusion equation with given diffusivity field k
and left-hand flux F.

ARGUMENTS: 
    xgrid = vector with equidistant grid points
        F = flux at left-hand boundary, k*du/dx = -F 
   source = source term, either a vector of values at points in xgrid
            or a constant
  rightbc = Dirichlet BC on right-hand boundary
Domain is given by xgrid (should be [0,1])
"""
def diffusioneqn(xgrid, F, k, source, rightbc):
    N = len(xgrid)
    h = xgrid[N-1] - xgrid[N-2]
    
    A = np.zeros((N-1, N-1))
    b = np.zeros(N-1)
    
    if isinstance(source, (int, float)):
        f = -source * np.ones(N-1)
    else:
        f = -source[:N-1]
    
    A -= np.diag(2*k[:-1] + k[1:] + np.insert(k[:-2],0,k[0]))
    A += np.diag(  k[:-2] + k[1:-1], 1)
    A += np.diag(  k[:-2] + k[1:-1],-1)
    A /= 2 * h**2
    
    A[0, 1] += k[0] / h**2
    b[0] = 2 * F / h
    
    b[N-2] = rightbc * (k[N-1] + k[N-2]) / (2 * h**2)
    
    uinternal = np.linalg.solve(A, f - b)
    usolution = np.append(uinternal, rightbc)
    
    return usolution


Let us first try to estimate the mean of the solution. To do so, we draw $n$ i.i.d. realizations of $(Y_1,Y_2,Y_3,Y_4,F)$, solve for $u(0.6,w)$ and take a sample average $\bar{U}_n = \dfrac{1}{n}\sum_{i=1}^n u(0.6,w_i)$. The 95% confidence interval then reads
$$
[\bar{U}_n - 1.96\frac{\hat{\sigma}_n}{\sqrt{n}}, \bar{U}_n + 1.96\frac{\hat{\sigma}_n}{\sqrt{n}}]
$$
where $\hat{\sigma}_n$ is the $n$-sample estimate of the standard deviation, which reads
$$
\hat{\sigma}_n = \sqrt{\dfrac{1}{n-1}\sum_{i=1}^n \left(u(0.6,w_i) - \bar{U}_n\right)^2}.
$$ 

In [3]:
xgrid = np.linspace(0,1,101)
sx = 5 
rbc = 1

def draw_diff(n):
    """ 
    return : 
        K : (n,4) - n independent diffusivity function on the grid
        Y : (n,4) - normal samples that define K
    """
    X = np.repeat(xgrid.reshape(1,-1), n, axis=0)
    K = np.zeros_like(X)
    Y = normal(-1, 1, size=(n,4))
    K[:,:25] = np.exp(Y[:,0:1].repeat(25,axis=1))
    K[:,25:50] = np.exp(Y[:,1:2].repeat(25,axis=1))
    K[:,50:75] = np.exp(Y[:,2:3].repeat(25,axis=1))
    K[:,75:] = np.exp(Y[:,3:].repeat(26,axis=1))
    return K

def draw_flux(n):
    return normal(-2, np.sqrt(.5), size=n)

def sde_mc(n, X=None):
    """ 
    MC simulation of sde at x = 0.6

    input:
        X : (n,5) - custom (Y,F) samples
            if None, just use the normal samples

    output: 
        u06 : (n,) simulation result
        S : (n,5) (Y,F) samples
    """
    u06 = []
    if X is None:
        K = draw_diff(n)
        F = draw_flux(n)
    else: 
        assert X.shape == (n,5)
        K = np.zeros((n,101))
        Y = X[:,:-1]
        F = X[:,-1]

        # define diffusivity from Y
        K[:,:25] = np.exp(Y[:,0:1].repeat(25,axis=1))
        K[:,25:50] = np.exp(Y[:,1:2].repeat(25,axis=1))
        K[:,50:75] = np.exp(Y[:,2:3].repeat(25,axis=1))
        K[:,75:] = np.exp(Y[:,3:].repeat(26,axis=1))


    for i in range(n):
        k = K[i,:]
        f = F[i]
        u = diffusioneqn(xgrid, f, k, sx, rbc)
        u06.append(u[60])

    return u06

def sde_mean(n, X=None):
    """ 
    MC estimate of sde mean at x = 0.6
    res : (3,)
        res[0] : left end of 95% CI
        res[1] : sample mean estimate
        res[2] : right end of 95% CI
        res[3] : standard error
    """
    res = np.zeros((4,))
    u06 = sde_mc(n, X=X)
    mu = np.mean(u06)
    res[1] = mu
    std = np.std(u06)
    res[0] = mu - 1.96*std/np.sqrt(n) # CI left
    res[2] = mu + 1.96*std/np.sqrt(n) # CI right
    res[3] = std/np.sqrt(n) # SE

    return res


We compute the mean estimate for $n$ ranging from 1000 to 10000, with a step size of 1000. The left and right ends of the 95% confidence interval (CI) are presented separately. The estimated standard error is also presented for reference. 

In [122]:
import pandas as pd
from IPython.display import display
nn = np.arange(1000,10001,1000)
resmat = np.zeros((4,len(nn)))
for count, n in enumerate(nn):
    resmat[:, count] = sde_mean(n)

d = {'n' : nn,
     'CI Left' : resmat[0,:], 
     'Sample Mean' : resmat[1,:],
     'CI Right' : resmat[2,:], 
     'SE' : resmat[3,:]}
df_mean = pd.DataFrame(d)
display(df_mean)

,n,CI Left,Sample Mean,CI Right,SE
0,1000,4.235773,4.536295,4.836818,0.153328
1,2000,4.234899,4.393714,4.552529,0.081028
2,3000,4.453821,4.597620,4.741418,0.073367
3,4000,4.364170,4.497932,4.631694,0.068246
4,5000,4.462437,4.574848,4.687259,0.057353
5,6000,4.500294,4.602541,4.704788,0.052167
6,7000,4.454526,4.545566,4.636605,0.046449
7,8000,4.543677,4.634582,4.725488,0.046380
8,9000,4.417137,4.495868,4.574598,0.040169
9,10000,4.479502,4.557361,4.635221,0.039724


## 2.1.(b)

We use the usual unbiased variance estimator, denoted as $\hat{\sigma}_n^2$. The standard error (SE) is $\frac{2\hat{\sigma}_n^4}{n-1}$. 

In [4]:
def sde_var(n, X=None):
    """ 
    MC estimate of sde variance at x = 0.6

    res : (2,)
        res[0] : estimated variacne
        res[1] : estimated standard error - 2\sigma^4 / (n-1)
    """
    res = np.zeros((2,))
    u06 = sde_mc(n, X=X)
    var = np.var(u06)
    res[0] = var
    res[1] = 2*var**2 / (n-1)

    return res

We compute the variance estimate for $n$ ranging from 1000 to 10000, with a step size of 1000. The estimated variance is presented along with the estimated standard error (SE). 

In [113]:
nn = np.arange(1000,10001,1000)
resmat = np.zeros((2,len(nn)))
for count, n in enumerate(nn):
    resmat[:,count] = sde_var(n)

d = {'n' : nn,
     'Variance' : resmat[0,:], 
     'SE' : resmat[1,:]}
df_var = pd.DataFrame(d)
display(df_var)

,n,Variance,SE
0,1000,12.276527,0.301728
1,2000,25.986476,0.675635
2,3000,14.233075,0.135099
3,4000,14.986581,0.112327
4,5000,15.020409,0.090263
5,6000,18.812687,0.117992
6,7000,16.365337,0.076532
7,8000,16.386115,0.067135
8,9000,16.103674,0.057635
9,10000,18.441665,0.068026


## 2.1.(c) Control Variate

Let $X = u(0.6,w)$ and $Y=(Y_1,Y_2,Y_3,Y_4,F)$. Define the control variate (CV) $Z = a^TY$ to be a linearization of $Y$. Let 
$$X^{cv}=X + c(Z-\mu_Z).$$
Then $\mathbb{E}[X^{cv}] = \mathbb{E}[X].$ As shown in class, the optimal $c$ has the form of 
$$
c^* = -\dfrac{\text{Cov}(X,Z)}{\text{Var}(Z)}.
$$ 
To estimate the variance of the CV estimator, we sample 50 independent $n$-sample sequences, compute 100 CV estimates, and estimate the variance from them.

In [114]:
from scipy.stats import multivariate_normal as mvn

def draw_Y(n):
    """ 
    return : (n,5) sample
    """
    mu = np.asarray([-1,-1,-1,-1,-2])
    s = np.asarray([1,1,1,1,0.5])
    S = np.diag(s)
    var = mvn(mean=mu, cov=S)
    return var.rvs(size=n)

def XYZ(n,a):
    a = a.reshape(-1,1)
    Y = draw_Y(n)
    Z = (Y@a).squeeze()
    X = sde_mc(n,X=Y)
    return X,Y,Z

def est_coeff(n,a):
    X,Y,Z = XYZ(n,a)
    X = np.asarray(X)
    XZ = X * Z 

    # cov(X,Z)
    mu = np.asarray([-1,-1,-1,-1,-2])
    mu_Z = np.dot(a,mu)
    cov_XZ = XZ.mean() - X.mean() * mu_Z

    # var(Z)
    s = np.asarray([1,1,1,1,0.5])
    var_Z = (s*np.power(a,2)).sum() # a'Sa 

    return -cov_XZ / var_Z     

def ctrl_var(n, a, c):
    """ 
    n : #samples for estimating mean
    nc : #samples for estimating c
    """
    X,Y,Z = XYZ(n,a)
    mu_vec = np.asarray([-1,-1,-1,-1,-2])
    mu_Z = np.dot(a,mu_vec)
    Xcv = X + c*(Z-mu_Z)
    res = np.zeros(4)

    mu = Xcv.mean()
    res[1] = mu
    std = np.std(Xcv)
    res[0] = mu - 1.96*std/np.sqrt(n) # CI left
    res[2] = mu + 1.96*std/np.sqrt(n) # CI right
    res[3] = std/np.sqrt(n) # SE
    return res

    
    

We present the experiment result for CV below. We choose $a = [0,0,0,-1,3]$. Note that this is purely based on heuristics. We compute the standard error (SE) and the 95% confidence interval. We keep $n$ the same as the previous experiments and compare the SE with the one for the vanilla Monte Carlo estimate.

In [154]:
# define linearization
np.random.seed(0)
a = [0.,0.,0.,-1.,3.]
a = np.asarray(a)

# estimate c
nc = 10000
c = est_coeff(nc,a)

nn = np.arange(1000,10001,1000)
# nn = [3000]
resmat = np.zeros((4,len(nn)))
for count, n in enumerate(nn):
    resmat[:,count] = ctrl_var(n,a,c)

d = {'n' : nn,
     'CI Left' : resmat[0,:], 
     'CV Mean' : resmat[1,:],
     'CI Right' : resmat[2,:], 
     'CV SE' : resmat[3,:],
     'Vanilla SE' : df_mean.get('SE')}
df_cv = pd.DataFrame(d)
display(df_cv)



,n,CI Left,CV Mean,CI Right,CV SE,Vanilla SE
0,1000,4.279330,4.532297,4.785264,0.129065,0.153328
1,2000,4.386884,4.534239,4.681595,0.075181,0.081028
2,3000,4.409243,4.529777,4.650311,0.061497,0.073367
3,4000,4.387149,4.489224,4.591299,0.052079,0.068246
4,5000,4.452020,4.552178,4.652337,0.051101,0.057353
5,6000,4.366468,4.445503,4.524537,0.040324,0.052167
6,7000,4.482305,4.562971,4.643636,0.041156,0.046449
7,8000,4.509084,4.580774,4.652464,0.036576,0.046380
8,9000,4.501744,4.572228,4.642712,0.035961,0.040169
9,10000,4.481907,4.545480,4.609052,0.032435,0.039724


From the result, we see that with this particular CV, we get at least 10% of variance reduction for those $n$ we have tested on compared to the vanilla Monte Carlo. In many cases, we get 20% of reduction. Note that our CV is chosen based on the trial-and-error heuristic with just a couple of experiments. With a more systematic and careful way of designing the CV, there must be a lot more space to improve the method. This demonstrates the superior performance and potential of CV. 

Below we consider two cases where $\mu_Z$ is very big or small. We first consider the big one by making the norm of $\mu_Z$ 10000 times bigger. We compare the SE of this amplified CV with the previous normal CV. 

In [155]:
# define linearization
np.random.seed(0)
a = [0.,0.,0.,-1.,3.]
a = np.asarray(a) * 10000

# estimate c
nc = 10000
c = est_coeff(nc,a)

nn = np.arange(1000,10001,1000)
# nn = [3000]
resmat = np.zeros((4,len(nn)))
for count, n in enumerate(nn):
    resmat[:,count] = ctrl_var(n,a,c)

d = {'n' : nn,
     'Big CV' : resmat[3,:],
     'Normal CV' : df_cv.get('CV SE')}
df_cv_big = pd.DataFrame(d)
display(df_cv_big)

,n,Big CV,Normal CV
0,1000,0.129065,0.129065
1,2000,0.075181,0.075181
2,3000,0.061497,0.061497
3,4000,0.052079,0.052079
4,5000,0.051101,0.051101
5,6000,0.040324,0.040324
6,7000,0.041156,0.041156
7,8000,0.036576,0.036576
8,9000,0.035961,0.035961
9,10000,0.032435,0.032435


Next, we consider the small one by making $\mu_Z$ 10000 times smaller and compare with the previous two different CV.

In [156]:
# define linearization
np.random.seed(0)
a = [0.,0.,0.,-1.,3.]
a = np.asarray(a) / 10000

# estimate c
nc = 10000
c = est_coeff(nc,a)

nn = np.arange(1000,10001,1000)
resmat = np.zeros((4,len(nn)))
for count, n in enumerate(nn):
    resmat[:,count] = ctrl_var(n,a,c)

d = {'n' : nn,
     'Big CV' : df_cv_big.get('Big CV'),
     'Normal CV' : df_cv.get('CV SE'),
     'Small CV' : resmat[3,:]}
df_cv_small = pd.DataFrame(d)
display(df_cv_small)

,n,Big CV,Normal CV,Small CV
0,1000,0.129065,0.129065,0.129065
1,2000,0.075181,0.075181,0.075181
2,3000,0.061497,0.061497,0.061497
3,4000,0.052079,0.052079,0.052079
4,5000,0.051101,0.051101,0.051101
5,6000,0.040324,0.040324,0.040324
6,7000,0.041156,0.041156,0.041156
7,8000,0.036576,0.036576,0.036576
8,9000,0.035961,0.035961,0.035961
9,10000,0.032435,0.032435,0.032435


We see that the variance stays exactly the same for these three cases. This is because the mean is subtracted from $Z$ in the CV estimator anyway, so as long as $a$ stays in the same direction, since this is a simple linear CV construction, the performance will be exactly the same with the same data.

## Monte Carlo Rare Event Simulation

Define random variable $X := \mathbb{1}\{u(0.6,w) > 40\}$. Note that
$$
\mathbb{E}\left[X\right] = \mathbb{P}\left[u(0.6,w) > u_0\right] = p
$$
where $\mathbb{1}\{\cdot\}$ is the indicator. Since $X$ is Bernoulli, we also have
$$
\text{Var}(X) = p(1-p).
$$
The Monte Carlo estimate of the mean and the standard deviation the estimate thereby reads
$$
\hat{p}_n = \dfrac{1}{n} \sum_{i=1}^n \mathbb{1}\{u(0.6,w_i) > 40\}
$$
and 
$$
\hat{\sigma}_n = \sqrt{\dfrac{\hat{p}_n(1-\hat{p}_n)}{n}}.
$$
We report the estimated relative error per sample (noted as REPS) $\dfrac{\sqrt{n}\hat{\sigma}_n}{\hat{p}_n} = \sqrt{\dfrac{1-\hat{p}_n}{\hat{p}_n}}$ accordingly.

In [53]:
def id_mc(n, X=None, IS=False, wx=None):
    """ 
    MC estimate of the indicator's mean and variance

    """
    if IS: 
        assert wx is not None
        assert X is not None 
        
    res = np.zeros((3,))
    u06 = sde_mc(n, X=X)
    u06 = np.asarray(u06)
    hx = np.where(u06 > 40, 1, 0)
    
    if IS:
        p = np.mean(wx * hx)
        wx2 = np.power(wx,2).sum()
        std = np.sqrt(p * (1-p) * wx2) / n
        reps = std * np.sqrt(n) / p
    else:
        p = np.mean(hx)
        std = np.sqrt(p*(1-p)/n)
        reps = np.sqrt((1-p)/p)
    
    res[0] = p # mean estimate
    res[1] = std # std estimate
    res[2] = reps # relative error per sample

    return res



We increase our number of samples to accomodate for the rareness. We choose $n$ to range from 2000 to 10000 with a step size of 1000. 

In [50]:
np.random.seed(0)
nn = np.arange(2000,10001,1000)
resmat = np.zeros((3,len(nn)))
for count, n in enumerate(nn):
    resmat[:,count] = id_mc(n)
d = {'n' : nn,
     r'p' : resmat[0,:], 
     r'STD' : resmat[1,:],
     r'REPS ' : resmat[2,:]}    
df_id = pd.DataFrame(d)
display(df_id)

,n,p,STD,REPS
0,2000,0.001000,0.000707,31.606961
1,3000,0.001667,0.000745,24.474477
2,4000,0.002000,0.000706,22.338308
3,5000,0.001200,0.000490,28.850188
4,6000,0.001167,0.000441,29.259919
5,7000,0.001143,0.000404,29.563491
6,8000,0.001000,0.000353,31.606961
7,9000,0.001222,0.000368,28.586392
8,10000,0.001300,0.000360,27.716976


## Cross-Entropy (CE) Method for Rare Event Simulation

Suppose after a good biasing distribution is constructed out of CE, the importance-sampling-based estimator of $p$ has the form of
$$
\hat{p}_n^{IS} = \dfrac{1}{n} \sum_{i=1}^n \mathbb{1}\{u(0.6,w_i) > 40\} \cdot W(w_i)
$$
where $W(w_i)$ are the weights. Then the standard deviation has the form of 
$$
\hat{\sigma}_n^{IS} = \dfrac{\left(\sum_{i=1}^n \hat{p}_n^{IS}(1-\hat{p}_n^{IS}) \cdot W(w_i)^2\right)^{1/2}}{n}
$$
and thus, the REPS takes the form of 
$$
\dfrac{\sqrt{n}\hat{\sigma}_n}{\hat{p}_n} = \sqrt{\dfrac{(1-\hat{p}_n^{IS})}{n\hat{p}_n^{IS}}\cdot \sum_{i=1}^n W(w_i)^2}.
$$

### Implementation of CE
The implementation of CE is presented below. The core method is `ce_train`. We provide implementations for 3 classes of biasing distributions
1. multivariate Gaussian with diagonal covariance matrix,
2. multivariate Gaussian with general SPD covariance matrix, and
3. other parameterized family of distributions,

and provide testing results for the first 2 cases. 

Note that for the first two cases, we have closed form update formula for the mean and the covariance matrix as presented in [the paper](https://mediatum.ub.tum.de/doc/1456133/document.pdf). The update rule is implemented in `normal_update`. Note that the formula will always give out a dense covariance matrix. Therefore, to ensure uncorrelation in case 1, we manually truncate the matrix to be diagonal. 

For the third case, we call the optimizer `scipt.optimize.minimize` to do the inner optimization for us. The loss function is implemented in `ce_loss`.



In [96]:
from scipy.optimize import minimize 
from scipy.stats import multivariate_normal as mvn

def init(option):
    if option=='normal uncorr':
        # mu = np.asarray([0.5, -1, -0.5, -5, -1])
        mu = -np.ones(5)
        s = np.eye(5).reshape(-1)
        theta = np.hstack((mu,s))
        return theta
    elif option=='normal corr':
        mu = -np.ones(5)
        s = np.eye(5).reshape(-1)
        theta = np.hstack((mu,s))
        return theta
    else:
        ValueError("{option} is not implemented!")

def sampler(theta, option='normal uncorr'):
    if option.split()[0]=='normal':
        mu = theta[:5]
        s = theta[5:]
        S = s.reshape(5,5)
        return mvn(mean=mu, cov=S)
    else:
        ValueError("{option} is not implemented!")

def sample_g(theta, n, option):
    var = sampler(theta, option)
    x = var.rvs(size=n)
    gx = var.pdf(x)
    return x, gx

def ce_loss(theta, x, px, gx_j, option):
    """ 
    gx_j : density for biasing distribution
    """
    u06 = sde_mc(n, X=x) # 
    u06 = np.asarray(u06)
    hx = np.where(u06 > 40, 1, 0)

    var = sampler(theta, option)
    gx = var.pdf(x) # density for current theta

    hpx = px * hx # unnormalized density of h(x)p(x)
    wx = hpx / gx_j # IS weight
    
    log_gx = np.log(gx)
    J = np.mean(wx*log_gx)
    return -J

def normal_update(theta, n, p, option):
    """ 
    closed form update of the normal density
    """
    x, gx = sample_g(theta, n, option)
    u06 = sde_mc(n, X=x)
    u06 = np.asarray(u06)
    hx = np.where(u06 > 40, 1, 0)
    px = p(x) # original density
    wx = px / gx # original density / biasing density
    hwx = hx * wx 
    hwx = hwx / hwx.sum() # normalize
    hwx = hwx.reshape(n,1) # reshape to broadcast
    mu = (x*hwx).sum(axis=0)
    assert mu.shape == (5,)

    mu_vec = mu.reshape(-1,1)
    S = np.zeros((5,5))
    for i in range(n):
        xi = x[i,:].reshape(-1,1)
        S += hwx.squeeze()[i] * (xi-mu_vec)@(xi-mu_vec).T
    
    if option.split()[1] == 'uncorr':
        # if uncorrelated, truncate
        S = np.diag(np.diagonal(S))
        
    return mu,S 

def ce_train(p, n, option='normal uncorr', maxits=1):
    """ 
    Cross Entropy method
    Goal: find a good biasing distribution for doing importance sampling
          a good biasing distribution should be similar to  |h(x)|p(x)

    p : input distribution
    """
    theta_j = init(option)

    if option.split()[0]=='normal':
        for _ in range(maxits): 
            mu, S = normal_update(theta_j, n, p, option)
            theta_j = np.hstack((mu, S.reshape(-1)))
        return theta_j
    
    # if biasing class is not normal, no closed form and use optimizer
    for _ in range(maxits):
        x, gx = sample_g(theta_j, n, option) # draw n samples from g_theta_i (n,5) (Y,F
        px = p(x) # (n,) known density of original distribution of x
        loss = lambda theta : ce_loss(theta, x, px, gx, option)
        res = minimize(loss, theta_j)
        theta_j = res.x
    
    return theta_j


We provide several numerical experiments to demonstrate the algorithm. Below we present the result for using uncorrelated multivariate normal distribution as the biasing distribution. To ensure the algorithm does not encounter overflow at the beginning, we need to have either a good initialization, or a large enough sample size (or both). Here, we let the sample size run from 2000 to 5000 with a step size of 1000. We select a simple initialization so that the initial mean vector is full of -1 and the covariance matrix is the identity matrix. We run the method for 10 iterations to obtain the parameter that defines the biasing distribution. With that parameter, we compute the importance-sampling-based estimate of $p$, its standard deviation, and the relative error per sample.

In [106]:
import pandas as pd 
from tqdm import tqdm 

np.random.seed(0)
nn = [2000, 3000, 4000, 5000]

# construct original distribution
mu = np.asarray([-1,-1,-1,-1,-2])
s = np.asarray([1,1,1,1,0.5])
S = np.diag(s)
var = mvn(mean=mu, cov=S)
p = lambda x : var.pdf(x)

# record result
resmat = np.zeros((3,len(nn)))

# ce config
option = 'normal uncorr'
maxits = 10
for count, n in tqdm(enumerate(nn)):
    theta = ce_train(p,n,option,maxits)
    x,gx = sample_g(theta, n, option)
    wx = p(x) / gx  # importance sampling weight
    resmat[:,count] = id_mc(n, X=x, IS=True, wx=wx)

d = {'n' : nn,
     r'p' : resmat[0,:], 
     r'STD' : resmat[1,:],
     r'REPS ' : resmat[2,:]}    
df_ce = pd.DataFrame(d)
display(df_ce)

4it [01:32, 23.16s/it]


,n,p,STD,REPS
0,2000,0.001117,0.000410,16.438551
1,3000,0.001041,0.000074,3.870158
2,4000,0.001090,0.000283,16.446418
3,5000,0.001112,0.000112,7.134117


Note that the result above is not the best we can get. We have tried different configurations for the method (initialization, number of iterations, etc.) and have obtained better result. But from the result above, we can already see that the REPS is decreased by a substantial amount compared to the vanilla Monte Carlo estimation.

Below we present the result for using correlated multivariate normal distribution as the biasing distribution. Everything else stays the same as the previous experiment.

In [104]:
resmat = np.zeros((3,len(nn)))

# ce config
option = 'normal corr'
maxits = 5
for count, n in tqdm(enumerate(nn)):
    theta = ce_train(p,n,option,maxits)
    x,gx = sample_g(theta, n, option)
    wx = p(x) / gx  # importance sampling weight
    resmat[:,count] = id_mc(n, X=x, IS=True, wx=wx)

d = {'n' : nn,
     r'p' : resmat[0,:], 
     r'STD' : resmat[1,:],
     r'REPS ' : resmat[2,:]}    
df_ce_corr = pd.DataFrame(d)
display(df_ce_corr)

4it [00:51, 12.89s/it]


,n,p,STD,REPS
0,2000,0.001049,0.000044,1.862702
1,3000,0.001023,0.000047,2.535822
2,4000,0.001117,0.000019,1.088884
3,5000,0.001099,0.000080,5.118815


With correlated Gaussian, we again see a substantial performance gain, in terms of the REPS quantity, compared to before. Interestingly, the REPS in this case is even a lot smaller than the uncorrelated Gaussian case. Intuitively, this makes sense because we are taking more information into account and allowing more complexity in the family of biasing distributions we are considering. However, there are many more factors to take into account before we draw any conclusion about this, such as initialization, randomness in the data, number of optimization iterations, etc. We leave the discussion here as it is beyond the scope of the assignment.